In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RSE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('refractory status epilepticus', 45),
 ('repeated sprint exercise', 8),
 ('redundant signal effect', 4),
 ('refractory se', 4),
 ('rsv stability element', 4),
 ('rna stability element', 3),
 ('radix scutellariae extract', 3),
 ('raspberry extract', 3),
 ('sachalinensis root', 3),
 ('relationship and sexuality education', 3),
 ('recto anal junction squamous epithelial', 3)]

In [14]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

KeyboardInterrupt: 

In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'radix scutellariae extract': 'ungrounded',
  'raspberry extract': 'ungrounded',
  'recto anal junction squamous epithelial': 'ungrounded',
  'redundant signal effect': 'ungrounded',
  'refractory se': 'ungrounded',
  'refractory status epilepticus': 'MESH:D013226',
  'relationship and sexuality education': 'ungrounded',
  'repeated sprint exercise': 'MESH:D000076962',
  'rna stability element': 'ungrounded',
  'rsv stability element': 'ungrounded',
  'sachalinensis root': 'ungrounded'},
 {'MESH:D013226': 'Status Epilepticus',
  'MESH:D000076962': 'High-Intensity Interval Training'},
 ['MESH:D000076962', 'MESH:D013226']]

In [32]:
grounding_map, names, pos_labels = grounding_map, names, pos_labels = [{'radix scutellariae extract': 'ungrounded',
  'raspberry extract': 'ungrounded',
  'recto anal junction squamous epithelial': 'ungrounded',
  'redundant signal effect': 'ungrounded',
  'refractory se': 'MESH:D013226',
  'refractory status epilepticus': 'MESH:D013226',
  'relationship and sexuality education': 'ungrounded',
  'repeated sprint exercise': 'MESH:D000076962',
  'rna stability element': 'ungrounded',
  'rsv stability element': 'ungrounded',
  'sachalinensis root': 'ungrounded'},
 {'MESH:D013226': 'Status Epilepticus',
  'MESH:D000076962': 'High-Intensity Interval Training'},
 ['MESH:D000076962', 'MESH:D013226']]

In [33]:
excluded_longforms = []

In [34]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [35]:
additional_entities = {}

In [36]:
unambiguous_agent_texts = {}

In [37]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [38]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [39]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [40]:
intersection1

[]

In [41]:
intersection2

[]

In [42]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [43]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [44]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 02:53:46] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 02:53:47] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9149019607843136 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [45]:
classifier.stats

{'label_distribution': {'ungrounded': 17,
  'MESH:D013226': 37,
  'MESH:D000076962': 3},
 'f1': {'mean': 0.914902, 'std': 0.063073},
 'precision': {'mean': 0.884028, 'std': 0.07432},
 'recall': {'mean': 0.95, 'std': 0.061237},
 'MESH:D013226': {'f1': {'mean': 0.961569, 'std': 0.03151},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.927778, 'std': 0.059187}},
 'ungrounded': {'f1': {'mean': 0.847778, 'std': 0.088304},
  'pr': {'mean': 0.816667, 'std': 0.152753},
  'rc': {'mean': 0.91, 'std': 0.111355}},
 'MESH:D000076962': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}}}

In [46]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [47]:
disamb.dump(model_name, results_path)

In [48]:
print(disamb.info())

Disambiguation model for RSE

Produces the disambiguations:
	High-Intensity Interval Training*	MESH:D000076962
	Status Epilepticus*	MESH:D013226

Class level metrics:
--------------------
Grounding                       	Count	F1     
              Status Epilepticus*	37	0.96157
                      Ungrounded	17	0.84778
High-Intensity Interval Training*	 3	    0.2

Weighted Metrics:
-----------------
	F1 score:	0.9149
	Precision:	0.88403
	Recall:		0.95

* Positive labels
See Docstring for explanation



In [49]:
model_to_s3(disamb)